<a href="https://colab.research.google.com/github/jun-wei-lin/NCHU/blob/main/AIoT-DA/HW4%3DAutoML%20and%20Ensemble/Hw4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**步驟 1: 確保環境設置完成**

In [ ]:
#安裝 PyCaret
!pip install pycaret

**步驟 2: 載入 Titanic 數據集**<br>
泰坦尼克數據集可以使用 Seaborn 載入，並進行必要的前處理：

In [ ]:
import seaborn as sns
import pandas as pd

# 加載 Titanic 數據集
df = sns.load_dataset('titanic')

# 刪除與預測無關的欄位
df = df.drop(['deck', 'embarked', 'alive', 'class'], axis=1)

# 填補缺失值
df['age'] = df['age'].fillna(df['age'].median())
df['embark_town'] = df['embark_town'].fillna(df['embark_town'].mode()[0])

# 編碼類別變量
df = pd.get_dummies(df, columns=['sex', 'embark_town'], drop_first=True)

# 查看數據集
df.head()


,survived,pclass,age,sibsp,parch,fare,who,adult_male,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,3,22.0,1,0,7.2500,man,True,False,True,False,True
1,1,1,38.0,1,0,71.2833,woman,False,False,False,False,False
2,1,3,26.0,0,0,7.9250,woman,False,True,False,False,True
3,1,1,35.0,1,0,53.1000,woman,False,False,False,False,True
4,0,3,35.0,0,0,8.0500,man,True,True,True,False,True


**步驟 3: 設置 PyCaret 環境**<br>
使用 PyCaret 的 setup 函數進行分類問題的設置：

In [ ]:
from pycaret.classification import *

# 設置 PyCaret 環境
clf1 = setup(
    data=df,
    target='survived',  # 目標變量
    session_id=123,     # 固定隨機種子
    use_gpu=True        # 使用 GPU（如果可用）
)


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

,Description,Value
0,Session id,123
1,Target,survived
2,Target type,Binary
3,Original data shape,"(891, 12)"
4,Transformed data shape,"(891, 14)"
5,Transformed train set shape,"(623, 14)"
6,Transformed test set shape,"(268, 14)"
7,Numeric features,5
8,Categorical features,1
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

**步驟 4: 比較模型性能** <br>
使用 compare_models 來比較多個分類模型，並選擇前 16 名模型：

In [ ]:
# 比較所有可用的模型
best_models = compare_models(n_select=16)

# 查看模型列表
best_models


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8216,0.8671,0.7487,0.7773,0.7607,0.6189,0.6213,0.2170
ridge,Ridge Classifier,0.8200,0.8666,0.7444,0.7772,0.7583,0.6153,0.6176,0.1610
lda,Linear Discriminant Analysis,0.8200,0.8663,0.7444,0.7772,0.7583,0.6153,0.6176,0.0950
gbc,Gradient Boosting Classifier,0.8186,0.8446,0.7027,0.7970,0.7447,0.6056,0.6100,0.3040
lightgbm,Light Gradient Boosting Machine,0.8153,0.8460,0.7277,0.7830,0.7509,0.6046,0.6089,0.1240
rf,Random Forest Classifier,0.8105,0.8464,0.7362,0.7672,0.7491,0.5971,0.5996,0.3680
nb,Naive Bayes,0.8008,0.8445,0.7569,0.7346,0.7415,0.5799,0.5852,0.1640
xgboost,Extreme Gradient Boosting,0.7977,0.8360,0.6904,0.7588,0.7199,0.5630,0.5666,0.1510
ada,Ada Boost Classifier,0.7960,0.8453,0.7406,0.7296,0.7325,0.5681,0.5707,0.1980
et,Extra Trees Classifier,0.7849,0.8201,0.7109,0.7258,0.7159,0.5431,0.5454,0.3280


Processing:   0%|          | 0/79 [00:00<?, ?it/s]

[LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                 max_iter=None, positive=False, random_state=123, solver='auto',
                 tol=0.0001),
 LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                            priors=None, shrinkage=None, solver='svd',
                            store_covariance=False, tol=0.0001),
 GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                            learning_rate=0.1, loss='log_loss', max_depth=3,
                            max_features=None, max_leaf_nodes=None,
                            min_impurity_d

**步驟 5: 微調模型**<br>
如果想進一步調整最佳模型，可以使用 tune_model：

In [ ]:
# 微調最佳模型
tuned_model = tune_model(best_models[0])  # 調整第一個模型的超參數

# 查看微調後的模型
tuned_model


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8413,0.8435,0.7083,0.8500,0.7727,0.6523,0.6587
1,0.8254,0.8344,0.7500,0.7826,0.7660,0.6268,0.6272
2,0.9365,0.9701,0.9167,0.9167,0.9167,0.8654,0.8654
3,0.8226,0.8818,0.6957,0.8000,0.7442,0.6094,0.6129
4,0.7419,0.7982,0.5833,0.7000,0.6364,0.4389,0.4433
5,0.7742,0.7796,0.7083,0.7083,0.7083,0.5241,0.5241
6,0.9032,0.9145,0.8750,0.8750,0.8750,0.7961,0.7961
7,0.7742,0.8311,0.7083,0.7083,0.7083,0.5241,0.5241
8,0.7581,0.8443,0.6250,0.7143,0.6667,0.4781,0.4807


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


LogisticRegression(C=7.863, class_weight={}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)